In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import plotly.graph_objects as go 
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from plotting import CandlePlot


In [3]:
df_raw = pd.read_pickle("../data/EUR_USD_H1.pkl")

In [4]:
df_raw.shape

(37219, 14)

In [5]:
df_an = df_raw.copy() #df_raw.iloc[-6000:].copy()
df_an.reset_index(drop=True, inplace=True)

In [6]:
df_an.shape


(37219, 14)

In [7]:
df_an = RSI(df_an)

In [8]:
df_an.tail ()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,RSI_14
37214,2021-12-30 19:00:00+00:00,1063,1.13058,1.13298,1.13043,1.13274,1.13052,1.13291,1.13036,1.13267,1.13065,1.13306,1.13049,1.13280,51.407635
37215,2021-12-30 20:00:00+00:00,1092,1.13272,1.13304,1.13185,1.13193,1.13264,1.13297,1.13177,1.13186,1.13279,1.13311,1.13192,1.13200,48.671929
37216,2021-12-30 21:00:00+00:00,388,1.13195,1.13276,1.13194,1.13258,1.13188,1.13269,1.13187,1.13249,1.13202,1.13283,1.13201,1.13267,50.928675
37217,2021-12-30 22:00:00+00:00,118,1.13252,1.13264,1.13222,1.13230,1.13204,1.13236,1.13182,1.13218,1.13299,1.13299,1.13236,1.13243,49.910668
37218,2021-12-30 23:00:00+00:00,507,1.13227,1.13258,1.13190,1.13224,1.13214,1.13251,1.13183,1.13216,1.13240,1.13266,1.13198,1.13231,49.681497


In [9]:
df_an = apply_patterns(df_an) 

In [10]:
df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()

In [11]:
df_an.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 'RSI_14',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
       'body_size_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev',
       'body_perc_prev_2', 'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR', 'EMA_200'],
      dtype='object')

In [12]:
our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',  'bid_c', 'ask_c', 'ENGULFING',
            'direction', 'EMA_200', 'RSI_14' ]

In [13]:
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)

In [14]:
df_slim.head()

,time,mid_o,mid_h,mid_l,mid_c,bid_c,ask_c,ENGULFING,direction,EMA_200,RSI_14
199,2016-01-19 07:00:00+00:00,1.08736,1.08764,1.08595,1.08652,1.08645,1.08660,False,-1,1.088334,36.239350
200,2016-01-19 08:00:00+00:00,1.08654,1.08846,1.08636,1.08846,1.08832,1.08861,True,1,1.088336,47.615598
201,2016-01-19 09:00:00+00:00,1.08844,1.08890,1.08702,1.08724,1.08699,1.08749,False,-1,1.088323,42.482293
202,2016-01-19 10:00:00+00:00,1.08728,1.08834,1.08662,1.08730,1.08722,1.08737,False,1,1.088312,42.808846
203,2016-01-19 11:00:00+00:00,1.08728,1.08732,1.08599,1.08630,1.08623,1.08637,True,-1,1.088289,38.849941


In [15]:
BUY =1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE        

In [16]:
df_slim["SIGNAL"] = df_slim.apply(apply_signal, axis=1)

In [17]:
df_slim["SIGNAL"].value_counts()

SIGNAL
 0    33779
-1     1635
 1     1606
Name: count, dtype: int64

In [18]:
LOSS_FACTOR = -1.0
PROFIT_FACTOR = 1.5


def apply_take_profit(row):
    if row.SIGNAL != NONE:
        return(row.mid_c - row.mid_o) * 1.5 + row.mid_c
    else:
        return 0.0

def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        return(row.mid_o - row.mid_c) * -1.0 + row.mid_c
    else:
        return 0.0
    


In [19]:
df_slim["TP"] = df_slim.apply(apply_take_profit, axis=1)
df_slim["SL"] = df_slim.apply(apply_stop_loss, axis=1)

In [20]:
df_slim[df_slim.SIGNAL==BUY].head()

,time,mid_o,mid_h,mid_l,mid_c,bid_c,ask_c,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,TP,SL
214,2016-01-19 22:00:00+00:00,1.09060,1.09143,1.09052,1.09140,1.09127,1.09153,True,1,1.088541,57.545307,1,1.092600,1.09220
216,2016-01-20 00:00:00+00:00,1.09112,1.09246,1.09100,1.09214,1.09206,1.09223,True,1,1.088610,59.970073,1,1.093670,1.09316
222,2016-01-20 06:00:00+00:00,1.09491,1.09635,1.09491,1.09546,1.09538,1.09554,True,1,1.088985,68.866726,1,1.096285,1.09601
224,2016-01-20 08:00:00+00:00,1.09398,1.09760,1.09397,1.09625,1.09619,1.09631,True,1,1.089121,67.087323,1,1.099655,1.09852
345,2016-01-27 09:00:00+00:00,1.08673,1.08764,1.08665,1.08726,1.08719,1.08732,True,1,1.085983,60.955020,1,1.088055,1.08779


In [21]:
df_plot = df_slim.iloc[70:100]
cp = CandlePlot(df_plot, candles=True)

trades = cp.df_plot[cp.df_plot.SIGNAL != NONE]

markers = ['mid_c', 'TP', 'SL']
marker_colors = ['#0000FF', '#00FF00', '#FF0000']

for i in range(3):
    cp.fig.add_trace(go.Scatter(
        x = trades.sTime,
        y = trades[markers[i]],
        mode = 'markers',
        marker=dict(color=marker_colors[i], size=12)
    ))

cp.show_plot(line_traces=["EMA_200"], sec_traces=['RSI_14'], height=250)

In [ ]:
df_results[df_results.SIGNAL==SELL].head()

In [23]:
class Trade:
    def __init__(self, row):
        self.running = True
        self.start_index = row.name
        self.start_price = row.mid_c
        self.trigger_price = row.mid_c
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 0
        
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
        
    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)
        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_l)
            elif row.mid_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_h)


In [24]:
open_trades = []
closed_trades = []

for index, row in df_slim.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.running == False:
            closed_trades.append(ot)
    open_trades = [x for x in open_trades if x.running == True]
    
    if row.SIGNAL != NONE:
        open_trades.append(Trade(row))

In [25]:
df_results = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [26]:
df_results.result.sum()

np.float64(-1468.5)

In [27]:
df_results.sort_values(by="start_index", inplace=True)

In [28]:
df_m5 = pd.read_pickle("../data/EUR_USD_M5.pkl")

In [29]:
df_m5.shape

(445038, 14)

In [30]:
df_m5.time.max()

Timestamp('2021-12-30 23:55:00+0000', tz='tzutc()')

In [31]:
df_raw.time.max()

Timestamp('2021-12-30 23:00:00+0000', tz='tzutc()')

In [32]:
from dateutil import parser

In [33]:
time_min = parser.parse("2021-12-15T10:00:00Z")
time_max = parser.parse("2021-12-15T11:00:00Z")
df_m5_s = df_m5[(df_m5.time>=time_min)&(df_m5.time<=time_max)]
df_raw_s = df_raw[(df_raw.time>=time_min)&(df_raw.time<=time_max)]

In [34]:
df_m5_s

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
441719,2021-12-15 10:00:00+00:00,208,1.12716,1.12745,1.12709,1.12734,1.12708,1.12738,1.12702,1.12727,1.12723,1.12752,1.12715,1.12742
441720,2021-12-15 10:05:00+00:00,132,1.12733,1.12754,1.12726,1.12746,1.12726,1.12747,1.12719,1.12739,1.12740,1.12760,1.12733,1.12752
441721,2021-12-15 10:10:00+00:00,166,1.12744,1.12752,1.12732,1.12746,1.12737,1.12745,1.12725,1.12739,1.12750,1.12760,1.12739,1.12753
441722,2021-12-15 10:15:00+00:00,138,1.12744,1.12770,1.12740,1.12761,1.12737,1.12763,1.12733,1.12754,1.12752,1.12776,1.12747,1.12768
441723,2021-12-15 10:20:00+00:00,252,1.12760,1.12762,1.12712,1.12712,1.12753,1.12756,1.12705,1.12705,1.12767,1.12769,1.12719,1.12719
441724,2021-12-15 10:25:00+00:00,198,1.12711,1.12726,1.12695,1.12695,1.12704,1.12719,1.12688,1.12688,1.12718,1.12733,1.12702,1.12702
441725,2021-12-15 10:30:00+00:00,142,1.12696,1.12698,1.12680,1.12696,1.12689,1.12691,1.12674,1.12690,1.12703,1.12706,1.12686,1.12703
441726,2021-12-15 10:35:00+00:00,121,1.12698,1.12712,1.12698,1.12700,1.12691,1.12705,1.12691,1.12693,1.12704,1.12718,1.12704,1.12706
441727,2021-12-15 10:40:00+00:00,206,1.12698,1.12707,1.12651,1.12656,1.12692,1.12700,1.12644,1.12649,1.12705,1.12714,1.12658,1.12663
441728,2021-12-15 10:45:00+00:00,165,1.12655,1.12668,1.12645,1.12654,1.12648,1.12661,1.12638,1.12648,1.12662,1.12675,1.12652,1.12661


In [35]:
df_raw_s

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
36941,2021-12-15 10:00:00+00:00,2038,1.12716,1.1277,1.12634,1.12673,1.12708,1.12763,1.12627,1.12666,1.12723,1.12776,1.12640,1.12680
36942,2021-12-15 11:00:00+00:00,2138,1.12674,1.1270,1.12626,1.12698,1.12667,1.12693,1.12619,1.12691,1.12681,1.12706,1.12631,1.12705
